In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
data_train = ImageDataGenerator(rescale = 1./255,
                                rotation_range = 7,
                                horizontal_flip = True,
                                shear_range = 0.2,
                                height_shift_range = 0.05,
                                zoom_range = 0.3)
data_valid = ImageDataGenerator(rescale = 1./255)

tam = 128

In [0]:
training = data_train.flow_from_directory('/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/dados/training',
                                            target_size = (tam, tam),
                                            batch_size = 10,
                                            class_mode = 'categorical')

validation = data_valid.flow_from_directory('/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/dados/validation',
                                          target_size = (tam, tam),
                                          batch_size = 10,
                                          class_mode = 'categorical')

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (tam, tam, 3), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, (3, 3), activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 10, activation = 'softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

model.summary()

In [0]:
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/melhor_peso.best.hdf5', save_best_only = True, verbose = 1)

history = model.fit(training,
                    batch_size = 8,
                    epochs = 250,
                    validation_data = validation,
                    verbose = 1,
                    validation_steps = 8,
                    callbacks=[checkpoint])

In [0]:
model_json = model.to_json()

with open('/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/modelo.json', 'w') as json_file:
    json_file.write(model_json)

In [0]:
data_test = ImageDataGenerator(rescale = 1./255)

testing = data_test.flow_from_directory('/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/dados/testing',
                                          target_size = (tam, tam),
                                          batch_size = 10,
                                          class_mode = 'categorical')

In [0]:
import matplotlib.pyplot as plt
plt.imshow(testing[0][0][0])

In [0]:
model.load_weights('/content/drive/My Drive/ColabNotebooks/Projetos Portifolio/Monkey Species Prediction/melhor_peso.best.hdf5')

(loss, accuracy) = model.evaluate(testing, batch_size = 16, verbose = 1)

In [0]:
print('Acuracia do Modelo: {:.2f}%'.format(accuracy * 100))

In [0]:
plt.figure(figsize=(12, 8))

freq = 1

loss = history.history['loss']    
val_loss = history.history['val_loss'] 
epochsloss = range(1, len(loss)+1)
epochsval_loss = [x*freq+1 for x in range(0, len(val_loss))]

plt.plot(epochsloss, loss, 'ro-', label='Training Loss')
plt.plot(epochsval_loss, val_loss, 'go-', label = 'Validation Loss')
plt.legend()

In [0]:
plt.figure(figsize=(12, 8))

loss = history.history['accuracy']      
val_loss = history.history['val_accuracy'] 
epochsloss = range(1, len(loss)+1)
epochsval_loss = [x*freq+1 for x in range(0, len(val_loss))]

plt.plot(epochsloss, loss, 'ro-', label='Training accuracy')
plt.plot(epochsval_loss, val_loss, 'go-', label = 'Validation accuracy')
plt.legend()